In [1]:
import numpy as np
import sympy as sy
import pandas as pd
from scipy.optimize import minimize

# Raceline Calculations
find the optimal turn radius and soves sor the ratios needed to acheve this

In [2]:
# symbols
t0, t1, red, s2, vt = sy.symbols("t0, t1, red, s2, vt")

## Constant input
input all constsants

given
- input reduction
- engine power
- engine speed
- mass
- wheel rad

assumsions
- mu: average coefficient of friction for rubber and asfault
- track width is constant
- can always use optimal line
- drivetrain mass is negleable for initial data
- tire rad is rad wheel + 0.8*130mm
- no final reduction now
- using a larger turn track
- since the track (see bellow) doesn't have a width, we find the track width using scale

In [3]:
# constants
g = 9.81

# constraints
# given
power = 24.4e3
init_red = 2.7
eng_sp = 6500  # rpm
m = 290

# calc
fin_red = 1
mu = 0.7
tire_rad = 0.3072
track_width = 26.6

data from https://virnow.com/wp-content/uploads/2011/11/VIR-planimetric-1.jpg
![Virginia Speedway](https://virnow.com/wp-content/uploads/2011/11/VIR-planimetric-1.jpg)
taking north corse

In [4]:
# list fof all turns
turn_array = [[215, 1644],
              [227, 120],
              [296, 270],
              [208, 679],
              [177, 298],
              [280, 907],
              [167, 137],
              [494, 358],
              [158, 100],
              [89, 106],
              [176, 309],
              [118, 1159],
              [173, 2889],
              [219, 444],
              [130, 277],
              [338, 2000],
              [328, 168],
              [284, 552],
              [228, 375],
              [315, 300],
              [251, 300],
              [105, 149],
              [97, 105],
              [90, 105],
              [94, 290],
              [226, 134],
              [162, 201],
              [220, 596],
              [337, 423],
              [175, 165],
              [299, 727],
              [92, 165],
              [200, 950],
              [693, 1375]]

calculating the race line is done so a driver can make use of the road width of
![Derivation](derivation.png)

In [5]:
def race_line(min_rad, max_rad, arc_ang):
    # calculates the raceline (max radius given width of track) for a turn
    cos_a = np.cos(np.deg2rad(arc_ang / 2))
    return (max_rad - min_rad * cos_a) / (1 - cos_a)


def find_r(arc_len, rad_cen):
    # if track is given as width and center radius insted of max, min radius

    ang = arc_len / rad_cen  # angle given arc
    min_rad = rad_cen - track_width
    max_rad = rad_cen + track_width
    ra = 12 * 0.0254  # ft to m
    return [min_rad * ra, max_rad * ra, np.rad2deg(ang)]

In [6]:
def by_ex(ratio_ls, cnt, defined_ratios=None):
    # calculates optimal ratios given number of gears, some preset ratios and optimal ratios for each turn
    def optimize(guess):
        # function for scipy to optomizes since we want the gear ratios where we have each turn be the closest to a gear
        defined_guess = np.sort(np.concatenate((guess, defined_ratios)))  # if some ratios are set
        cost = 0
        for turn in ratio_ls:
            # minds the optimal ratio for each turn, scypi will find min diff
            cost += np.min(np.abs(defined_guess - turn))
        return cost

    if defined_ratios:
        cnt -= len(defined_ratios)
    else:
        defined_ratios = []
    ratio_ls.sort()
    out_r = np.linspace(ratio_ls[0], ratio_ls[-1], cnt)  # initial guess
    result = minimize(optimize, out_r, options={'disp': False})
    return result.x  # function can return iterations, only need output

In [7]:
# turn radius array [min, max, arc_angle] or [arc_len, center rad]
if len(turn_array[0]) == 2:  # see if need to convert to min max
    turn_array_norm = [find_r(*turn_n) for turn_n in turn_array]
else:
    turn_array_norm = turn_array

rad_array = np.array(turn_array_norm)
rad_array = np.hstack((rad_array, np.zeros([rad_array.shape[0], 3])))  # shape correctly

In [8]:
# full drivetrain redux
total_r = red * init_red * fin_red

## iterations
loops trough each turn and calculates their ratios based on the above functions

In [ ]:
# loops through each turn and calculates all values
for i in range(rad_array.shape[0]):
    rad = race_line(*rad_array[i, :3])  # finds the optimale raceline for each turn
    # all eqs in the form x-y = 0,

    t_eq = [t0 - power / eng_sp,  # find initial torque
            s2 - eng_sp / total_r,  # find output speed (solving for red)
            vt - np.sqrt(g * mu * rad),  # vt from turn (acc_n*m = v^2/r, and acc_n*m <= Fric_F)
            vt - tire_rad * s2 * np.pi / 30  # vt = out speed * r, convering rpm to rad/s
            ]

    sol = sy.nonlinsolve(t_eq, [red, s2, t0, vt])  # soving system
    rad_array[i, 3:] = [rad, sol.args[0][3], sol.args[0][0]]  # used to get v and gear ratios

## turn data
the data for each turn is

In [9]:
# creats dataframe of values for pretty print or saving
df = pd.DataFrame(rad_array, columns=["min", "max", "deg", "race", "v", "R"])
print(df)

          min        max         deg          race           v         R
0   492.98352  509.19888    7.493061   8080.471900  235.560185  0.328774
1    28.46832   44.68368  108.384516     67.547810   21.537196  3.595922
2    74.18832   90.40368   62.813151    184.866571   35.629745  2.173635
3   198.85152  215.06688   17.551579   1583.943679  104.292575  0.742585
4    82.72272   98.93808   34.031386    453.144815   55.783021  1.388345
5   268.34592  284.56128   17.687782   1632.230242  105.870322  0.731518
6    33.64992   49.86528   69.842301    123.705543   29.145943  2.657182
7   101.01072  117.22608   79.061774    171.907326   34.358225  2.254077
8    22.37232   38.58768   90.527332     77.125329   23.013466  3.365250
9    24.20112   40.41648   48.106834    210.940575   38.059545  2.034866
10   86.07552  102.29088   32.634489    488.649299   57.927150  1.336956
11  345.15552  361.37088    5.833393  12862.529500  297.198570  0.260587
12  872.45952  888.67488    3.431004  37051.071771 

## optimization
thus we can culculate the output
using scipy optimize

In [10]:
# calculates optimal gear ratios based on turns
rat = by_ex(list(df['R']), 4)
print(f'\nOptimal ratios for track are: {rat}\n')





Optimal ratios for track are: [0.41197844 1.14765844 2.07078981 3.36449413]

